In [1]:
# libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.integrate import simps
import scipy.constants as cte
from scipy.sparse import diags

In [2]:
# grandezas de interesse em unidades atomicas
au_l = cte.value('atomic unit of length')
au_t = cte.value('atomic unit of time')
au_e = cte.value('atomic unit of energy')

# outras relacoes de interesse
ev = cte.value('electron volt')
au2ang = au_l / 1e-10
au2ev = au_e / ev

In [ ]:
# unidades do problema
E_0 = 150.0 # eV
L = 100.0 # angstron
sigma_x = 1.0 # angstron
x_0 = -20.0 # angstron
dt = dt_0 = 1e-15 # s

# unidades atomicas
E_0_au = E_0 / au2ev
L_au = L / au2ang
sigma_x_au = sigma_x / au2ang
x_0_au = x_0 / au2ang
dt_au = dt / au_t
k_0_au = np.sqrt(2 * E_0_au)

# salva os valores onde o algoritmo converge
par_convergentes = []

# divisor esperto (?)
de = lambda x: 2 if int((x/(10**(int(np.log10(x))-1)))%2) == 0 else 5

for N in [2**n for n in range(8,12)]:
    
    dt = dt_0
    
    # malha espacial
    x_au = np.linspace(-L_au/2, L_au/2, N)
    dx_au = x_au[1] - x_au[0]
    # diferencas finitas
    alpha = 1j / (2 * dx_au ** 2)
    beta = - 1j / (dx_au ** 2)
    diagonal_1 = [beta] * N
    diagonal_2 = [alpha] * (N - 1)
    diagonais = [diagonal_1, diagonal_2, diagonal_2]
    M = diags(diagonais, [0, -1, 1]).toarray()
    
    while True:
        #dt /= 10
        dt /= de(dt)
        dt_au = dt / au_t
        
        # pacote de onda
        PN = 1/(2*np.pi*sigma_x_au**2)**(1/4)
        psi = PN*np.exp(1j*k_0_au*x_au-(x_au-x_0_au)**2/(4*sigma_x_au**2))
        A0 = (simps(np.conjugate(psi)*psi,x_au)).real
        x_f_au = x_0_au 
        
        funcionou = True
        contador = 0
        norma = 100
        skewness = 0
        
        while x_f_au < -x_0_au:
            try:
                psi += dt_au * M.dot(psi)
                contador += 1
                if contador % 10 == 0:
                    psis = np.conjugate(psi)
                    A = (simps(psis*psi,x_au)).real
                    norma = 100 * A / A0
                    if np.abs(norma - 100) > 5:
                        funcionou = False
                        break
                    
                    x_f_au = xm1 = (simps(psis* x_au * psi,x_au)).real / A
                    xm2 = (simps(psis* x_au**2 * psi,x_au)).real / A
                    xm3 = (simps(psis* x_au**3 * psi,x_au)).real / A
                    
                    sigma = np.sqrt(np.abs(xm2 - xm1**2))
                    skewness = gamma = (xm3 - 3*xm1*sigma**2-xm1**3)/sigma**3
                    if np.abs(gamma) > 0.1:
                        funcionou = False
                        break
            except:
                funcionou = False
                break
                
        parametros = (N, dt, norma, skewness, contador)
        if funcionou:
            par_convergentes.append(parametros)
            break
        print("Estouro: N = {}, dt={:.2e} s, A = {:.2f}, G = {:.2f}, contador = {}".format(*parametros))
print(par_convergentes)

Estouro: N = 256, dt=5.00e-16 s, A = 533291951356941807303184333738195550208.00, G = 0.00, contador = 10
Estouro: N = 256, dt=1.00e-16 s, A = 5870033694535595327488000.00, G = 0.00, contador = 10
Estouro: N = 256, dt=5.00e-17 s, A = 6505731819343609856.00, G = 0.00, contador = 10
Estouro: N = 256, dt=1.00e-17 s, A = 3520470.93, G = 0.00, contador = 10
Estouro: N = 256, dt=5.00e-18 s, A = 4182.29, G = 0.00, contador = 10
Estouro: N = 256, dt=1.00e-18 s, A = 119.19, G = 0.00, contador = 10
Estouro: N = 256, dt=5.00e-19 s, A = 109.23, G = 0.00, contador = 20
Estouro: N = 256, dt=1.00e-19 s, A = 105.08, G = 0.02, contador = 280
Estouro: N = 256, dt=5.00e-20 s, A = 105.03, G = 0.03, contador = 1110
Estouro: N = 256, dt=1.00e-20 s, A = 103.99, G = -0.10, contador = 22130
Estouro: N = 256, dt=5.00e-21 s, A = 101.98, G = -0.10, contador = 44260
Estouro: N = 256, dt=1.00e-21 s, A = 100.39, G = -0.10, contador = 221300
Estouro: N = 256, dt=5.00e-22 s, A = 100.20, G = -0.10, contador = 442600
Est